In [20]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.append("../")
sys.path.append("../site")

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [4]:
import re
import stanza
from collections import Counter
import itertools
import datasets
import pickle
from transformers import AutoModelWithLMHead, AutoTokenizer
stanza.download('en')  

2020-05-01 19:16:20 INFO: Downloading default packages for language: en (English)...
2020-05-01 19:16:21 INFO: File exists: /home/tdimson/stanza_resources/en/default.zip.
2020-05-01 19:16:24 INFO: Finished downloading models and saved to /home/tdimson/stanza_resources.


In [14]:
def print_words(words, f):
    for word in words:
        word_str = [word.word]
        if word.pos:
            word_str.append(f"/{word.pos}/")
        if word.topic:
            word_str.append(f"[{word.topic}]")
        print(" ".join(word_str), file=f)
        print(f"\t{word.definition}{' |n| ' if word.example is None else ''}", file=f)
        if word.example:
            print(f"\t\"{word.example}\"{' |e|' if word.from_example_expansion else ''}", file=f)

        print("", file=f)

In [15]:
nlp = stanza.Pipeline(lang='en', processors='tokenize,mwt,pos')
tokenizer = AutoTokenizer.from_pretrained("gpt2")
tokenizer.add_special_tokens(datasets.SpecialTokens.special_tokens_dict())
blacklist = datasets.Blacklist.load("../data/blacklist.pickle")
model = AutoModelWithLMHead.from_pretrained("../models/en_dictionary_parsed_lr_00001/checkpoint-100000").to("cuda:0")

2020-05-01 19:17:45 WARNING: Can not find mwt: default from official model list. Ignoring it.
2020-05-01 19:17:45 INFO: Loading these models for language: en (English):
| Processor | Package |
-----------------------
| tokenize  | ewt     |
| pos       | ewt     |

2020-05-01 19:17:45 INFO: Use device: gpu
2020-05-01 19:17:45 INFO: Loading: tokenize
2020-05-01 19:17:45 INFO: Loading: pos
2020-05-01 19:17:46 INFO: Done loading processors!


'<|bod|>the feeling of falling perpetually into the ground.<|bd|>'

In [18]:
words, stats = datasets.ParsedDictionaryDefinitionDataset.generate_words(
    tokenizer, model,
    num=500,
    max_iterations=40, 
    blacklist=blacklist, 
    do_example_expansion=True, 
    generation_args=dict(
        top_k=300,
        num_return_sequences=100,
        max_length=400,
        do_sample=True,
    ),
    expansion_generation_overrides=dict(
        top_k=50,@
        num_return_sequences=10,
        do_sample=True,
    ),
    num_expansion_candidates=10,
    filter_proper_nouns=True,
)

print(stats)
print()
print_words(words, sys.stdout)

iterations=40 | items_considered 1.00@4000, failed_match 0.01@21, blacklist_filtered 0.82@3294, seen_filtered 0.00@3, proper_noun_filtered 0.04@149, example_filtered 0.00@0, example_expansions 0.12@460, example_expansion_success 0.11@422, example_expansion_hail_maries 0.00@0, user_filtered 0.00@0, returned 0.12@495

coptosis /noun/
	a condition in which the skin or a part of the body is exposed to hot air at high altitude for over several months then gradually developing into shortening of the skin.
	"over 50 percent of deaths follow cardiac Coptosis" |e|

inhominemesis /noun/
	the addition of an exogenous antigen in an embryo after pregnancy to a natural antigen that has already been fertilized.
	"the use of artificial inhominemesis is highly controversial" |e|

phobius /noun/
	an upper class group comprising the Permianians.
	"the phobius will not tolerate failure under the thumb of their own political masters" |e|

phosphotransferase /noun/
	one or more molecules released from a vir

In [22]:
from words import WordIndex, Word

In [24]:
wi = WordIndex(
    [
        Word(
            word=w.word,
            definition=w.definition,
            pos=w.pos,
            topic=w.topic,
            example=w.example,
        ) for w in words
    ]
)
wi.dump("../site/data/words.json")